In [57]:
import pandas as pd
from pymongo import MongoClient

In [58]:
actions = pd.read_excel(r"Combined.xlsx")
actions.head(1)

,Strategic interventions,Action,Lead Responsible,Partners,Timeframe,Start Time Frame,End Time Frame,Topics,File Name,Target,Performance indicators
0,NaN,Provide potable water in all learning centres ...,"DES, pre-schools, schools, colleges, universit...",NaN,2016–2025,2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...,NaN,NaN


In [59]:
actions.drop(columns = ["Strategic interventions", "Partners", "Timeframe", "Target", "Performance indicators"], inplace = True)
actions = actions.dropna(subset = ["Lead Responsible"])
actions.head(1)

,Action,Lead Responsible,Start Time Frame,End Time Frame,Topics,File Name
0,Provide potable water in all learning centres ...,"DES, pre-schools, schools, colleges, universit...",2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...


In [60]:
def create_list(s):
    return [item.strip() for item in s.split(",")]

actions["Lead Responsible"] = actions["Lead Responsible"].apply(create_list)
actions.head(1)

,Action,Lead Responsible,Start Time Frame,End Time Frame,Topics,File Name
0,Provide potable water in all learning centres ...,"[DES, pre-schools, schools, colleges, universi...",2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...


In [61]:
actions = actions.explode("Lead Responsible").reset_index(drop = True)
actions.head(5)

,Action,Lead Responsible,Start Time Frame,End Time Frame,Topics,File Name
0,Provide potable water in all learning centres ...,DES,2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...
1,Provide potable water in all learning centres ...,pre-schools,2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...
2,Provide potable water in all learning centres ...,schools,2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...
3,Provide potable water in all learning centres ...,colleges,2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...
4,Provide potable water in all learning centres ...,universities.,2016,2025,"['water', 'potable water', 'community']",A Healthy Weight for Ireland - Obesity Policy ...


In [62]:
unique_topics = actions['Topics'].unique()
print(unique_topics)

["['water', 'potable water', 'community']"
 "['rural development', 'community', 'economic viability']"
 "['rural development', 'community']"
 "['community', 'rural development', 'social sustainability', 'society']"
 "['rural development', 'community', 'social sustainability', 'society']"
 "['water', 'waterways', 'community']"
 "['carbon', 'rural development', 'community', 'social sustainability']"
 "['rural development', 'community', 'society']"
 "['rural development', 'community', 'social sustainability', 'economic viability']"
 "['community', 'social sustainability', 'economic viability']"
 "['community', 'social sustainability', 'society']"
 "['social sustainability', 'community']"
 "['economic viability', 'rural development', 'social sustainability']"
 "['community']"
 "['community', 'social sustainability', 'rural development']"
 "['rural development', 'economic viability', 'community', 'society']"
 "['economic viability', 'community', 'rural development']"
 "['community', 'econom

In [63]:
def insert_dataframe_into_mongodb(mongo_uri, db_name, collection_name, df):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]

    data = df.to_dict(orient="records")

    if data:
        result = collection.insert_many(data)
        print(f"{len(result.inserted_ids)} documents inserted successfully.")
        return len(result.inserted_ids)
    else:
        print("No data to insert.")
        return 0

In [64]:
insert_dataframe_into_mongodb('mongodb://localhost:27017/', 'extraction', 'action_plans', actions)

255 documents inserted successfully.


255